## Train

In [12]:
import bentoml

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

In [13]:
model = RandomForestClassifier()
iris = load_iris()

In [14]:
X = iris.data[:, :4]
Y = iris.target
model.fit(X, Y)

RandomForestClassifier()

In [15]:
bento_model = bentoml.sklearn.save_model('randomforest', model)
print(f"Model saved: {bento_model}")

Model saved: Model(tag="randomforest:zgruojud3cjxl3eo")


## Testing

In [16]:
clf = bentoml.sklearn.get('randomforest:latest').to_runner()
clf.init_local()
print(clf.predict.run([[2,3,4,5]]))

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


[2]


## Service

In [17]:
%%writefile -a service.py
import bentoml
import numpy as np
from bentoml.io import NumpyNdarray

clf = bentoml.sklearn.get('randomforest:latest').to_runner()

service = bentoml.Service(
    "randomforest", runners=[clf]
)

# Create an API function
@service.api(input=NumpyNdarray(), output=NumpyNdarray())
def predict(input_series: np.ndarray) -> np.ndarray:

    result = clf.predict.run(input_series)
    
    return result

Appending to service.py


In [11]:
%%writefile -a bentofile.yaml
service: "service.py:service"  # Same as the argument passed to `bentoml serve`
labels:
    owner: bentoml-team
    project: gallery
include:
- "*.py"  # A pattern for matching which files to include in the bento
python:
    packages:  # Additional pip packages required by the service
    - scikit-learn
    - pandas

Writing bentofile.yaml
